In [7]:
#Import libraries
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [45]:
#Import list of job openings
#Depending on the page being scraped, comment out the page number
jobpage = "https://euraxess.ec.europa.eu/jobs/search?keywords=pharmaceutical" + "&page=1"
page = urlopen(jobpage)
soup = BeautifulSoup(page, "lxml")

In [43]:
#Parse the webpage into various fields
def addtable(isoup, df, link):
    jobtitle = [x.text.strip() for x in isoup.find_all("h1", class_="head-title")]
    body = [x.text.strip() for x in isoup.find_all("div", class_="longtext field-body")]
    organization = [x.text for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-company-institute inline")]
    field = [[y.strip() for y in x.text.strip().split(" \n\n")] for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-research-field inline")]
    profile = [[y.strip() for y in x.text.strip().split(" \n\n")] for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-research-profile inline")]
    location = [x.text.strip().split(" \n\n") for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-country inline")]
    city = [x.text.strip() for x in isoup.find_all("div", class_="value field-city")]
    addinfo = [x.text.strip() for x in isoup.find_all("div", class_="col-xs-12 field-additional-information")]
    reqt = [x.text.strip() for x in isoup.find_all("div", {"id":"bootstrap-fieldgroup-accordion-item--requirements-1"})]
    timex = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    newrow = {"Jobtitle":jobtitle, 
              "Organization":organization, 
              "Field":field,
              "Profile":profile, 
              "Location":location, 
              "City":city, 
              "Requirements":reqt, 
              "Description":body, 
              "Misc":addinfo,
             "Link": link,
             "Time": timex}
    return newrow

In [48]:
#Parse each job opening result into a pandas dataframe
for x in soup.find_all("div", class_="col-sm-12 col-md-6"):
    jobscsv = pd.read_pickle("jobs.pkl")
    print(x.a["href"])
    if x.a["href"] not in jobscsv["Link"].unique():
        indpage=urlopen("https://euraxess.ec.europa.eu" + str(x.a["href"]))
        isoupx = BeautifulSoup(indpage, "lxml")
        newrow = addtable(isoupx, jobscsv, str(x.a["href"]))
        jobscsv =jobscsv.append(newrow, ignore_index=True)
        jobscsv.to_pickle("jobs.pkl")
        jobscsv.to_csv("jobs.csv")
        for i in range(60*20):
            time.sleep(1)

/jobs/178426
/jobs/178432


KeyboardInterrupt: 

In [100]:
import six
from ast import literal_eval

In [132]:
#Sample parsing
joblist = pd.read_pickle("jobs.pkl")
joblist["Link"] = joblist["Link"].str.extract("([1-9]+)")
joblist["Organization"] = joblist["Organization"].apply(lambda x: literal_eval(str(x))[0].strip())
joblist["Jobtitle"] = joblist["Jobtitle"].apply(lambda x: literal_eval(str(x))[0].strip())
joblist["Field"] = joblist["Field"].apply(lambda x: literal_eval(str(x))[0])
joblist["Profile"] = joblist["Profile"].apply(lambda x: literal_eval(str(x))[0])
joblist["Location"] = joblist["Location"].apply(lambda x: literal_eval(str(x))[0][0])
joblist["City"] = joblist["City"].apply(lambda x: literal_eval(str(x))[0])
joblist["Requirements"] = joblist["Requirements"].apply(lambda x: literal_eval(str(x))[0] if len(literal_eval(str(x)))>0 else x)
joblist["Description"] = joblist["Description"].apply(lambda x: literal_eval(str(x))[0] if len(literal_eval(str(x)))>0 else x)
joblist["Misc"] = joblist["Misc"].apply(lambda x: literal_eval(str(x))[0] if len(literal_eval(str(x)))>0 else x)
joblist

C:\Users\ara730\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


,Link,Jobtitle,Organization,Field,Profile,Location,City,Requirements,Description,Misc,Time
0,175426,Cell Biologist Oncology,Astex Pharmaceuticals,"[Biological sciences, Literature]","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,[],Job Title: Cell Biologist Oncology (Ref CBO/01...,Additional commentsContract\nWeb site for addi...,NaN
1,175428,Neuroinflammation / Neuroimmunologist,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Medical scien...","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,[],Job Title: Neuroinflammation / Neuroimmunologi...,Additional commentsPermanent\nWeb site for add...,NaN
2,175429,Protein Trafficking Expert,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Neurosciences...","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,[],Job Title: Protein Trafficking Expert (Ref: PT...,Additional commentsPermanent\nWeb site for add...,NaN
3,175427,Translational Biologist Oncology,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Literature]","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,[],Job Title: Translational Biologist Oncology (R...,Additional commentsContract\nWeb site for addi...,NaN
4,179884,Early Stage Researcher (for a PhD study) for a...,Janssen Pharmaceuticals,"[Mathematics ? Statistics, Psychological scien...",[First Stage Researcher (R1)],Sweden,Solna,Offer Requirements\n\nREQUIRED EDUCATION LEVEL...,Janssen Pharmaceuticals and Karolinska Institu...,Eligibility criteriaCandidates will be require...,NaN
5,178427,Bioinformatics Scientist,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Neurosciences]","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,Offer RequirementsSkills/QualificationsPostgra...,Astex Pharmaceuticals is a world leader in inn...,Additional commentsContract\nWeb site for addi...,Timestamp: 2017-02-27 10:50:08
6,178429,Biophysical Scientist,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Neurosciences]","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,Offer RequirementsSkills/QualificationsPostgra...,Astex Pharmaceuticals is a world leader in inn...,Additional commentsContract\nWeb site for addi...,Timestamp: 2017-02-27 11:00:09
7,178423,Cell Biologists Oncology,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Neurosciences]","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,Offer RequirementsSkills/QualificationsPostgra...,Astex Pharmaceuticals is a world leader in inn...,Additional commentsContract\nWeb site for addi...,Timestamp: 2017-02-27 11:10:11
8,178428,Cryo-Electron Microscopy Scientist,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Neurosciences]","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,Offer RequirementsSkills/QualificationsPostgra...,Astex Pharmaceuticals is a world leader in inn...,Additional commentsContract\nWeb site for addi...,Timestamp: 2017-02-27 11:21:46
9,178433,Neurobiology Postdoc,Astex Pharmaceuticals,"[Biological sciences, Chemistry, Neurosciences]","[First Stage Researcher (R1), Recognised Resea...",United Kingdom,Cambridge,Offer RequirementsSkills/QualificationsPostgra...,Astex Pharmaceuticals is a world leader in inn...,Additional commentsContract\nWeb site for addi...,Timestamp: 2017-02-27 11:25:53
